# Scatter

An opinionated package for creating RPCs in Python with type validation and version controlled functions. I have a working version of this for FastAPI as well, but this is cooler.

In [ ]:
from scatter.brew.core import make_callable, current_version, rollback, scatter, vaporize

num_1 = 42
num_2 = 42 * 24

In [ ]:
@scatter
def add(a: int, b: int) -> int:
    return a + b

^ The function will be "scattered" on the server and can now be called multiple times by multiple clients in parallel WITH type validation. And we can now shutdown the notebook if needed.

In [ ]:
print("Current function version:", current_version("add"))

callable_func = make_callable("add")

print("Function call result:", callable_func(num_1, num_2))

^ We call `make_callable` with the function name and we'll get a callable object with the same signature as the original function - thus will do type validation on the client side itself.

### ^ I don't want to write protobuf files or any other IDL (Interface Definition Language).

In [ ]:
@scatter
def add(a: int, b: int, c: int) -> int:
    return a + b + c

^ Let's change the function signature and scatter again with the same function name, this will update the old function with the new signature.

In [ ]:
print("Version after updating signature:", current_version("add"))

In [ ]:
# Updated function

callable_func = make_callable("add")
print("Updated function call result:", callable_func(num_1, num_2))  # This will fail since it now expects 3 arguments
# print("Updated function call result:", callable_func(num_1, num_2, num_1))

Rolling back to the previous version of the function:

In [ ]:
# Rollback

rollback("add")
print("Version after rollback:", current_version("add"))

In [ ]:
# Calling rolledback function
callable_func = make_callable("add")
print("Rolled back function call result:", callable_func(num_1, num_2))

Deleting the function:

In [ ]:
print("Deleting the function...")
vaporize("add")

Some opinions:

- WHY DO I NEED TO RESTART THE ENTIRE APP TO ADD A NEW FUNCTION OR EDIT AN EXISTING ONE? (in a lot of cases I shouldn't have to)

- Not a fan of classes for RPCs. I prefer functions.

- Stateless is better than stateful. A philosphy which is best suited when writing APIs. That said, I do see the value in stateful RPCs.

- No reason to not have type validation always on.

- I want to rollback the functions to a previous version if I need to.

- Would like to avoid having to use the CLI as it breaks my immersion when coding and is a bit of a barrier to start using any new package.

- No storage of results, args, or kwargs - it's not the server's responsibility to store this information.


- I shouldn't need to write all my functions at the time when I start the app - dynamic addition/deletion/updation of functions should be possible. Let's say a new node joins and they want to add a new function to the server, they should be able to do so without restarting the server.

- Simplicity is key

- ⁠Although [P]ython [O]bject [O]riented [P]rogramming is POOP, it does come in handy every once in a while

- Migration should be simplified using a "Set" of related functions that can be added/removed/updated in one go - will be added soon.


Cool packages used:
- `hapless` - for background process management and CLI
- `typer` - to write a good looking rich based CLI
- `msgspec` - for type validation, serialization and deserialization with `msgpack` -> which is almost universally seen as a faster format - all of this is done faster than pydantic
- `zeroapi` - for the server and client communication using RPC and ZeroMQ - a brokerless message queue protocol
- `cloudpickle` - to serialize/deserialize functions AND ONLY THE UNSUPPORTED ARBITRARY TYPES - all the supported types are already handled by `msgspec`
- `redis` or `diskcache` - backend to store the serialized functions in cache for really fast retrieval and manual mechanism of versioning the functions